# INITIALISATIONS

This program loads in .npy files produced by file-processing.ipynb and performs CNN classification.

In [1]:
# Initializing TF and Keras
from __future__ import division
import sys

# Import ray tune (hyperparameter tuning)
import ray
from ray import tune        
from ray.tune.util import pin_in_object_store #, get_pinned_object

# Clear logs from previous runs
!rm -rf ./tensorboard-logs/

# Notes ############################################################################
#
# 1.
#     Normal tf import is disabled to solve ray.tune pickling error
#     import tensorflow as tf 
# 2.
#     To automatically view training result with tensorboard, use 
#         tensorboard --logdir ~/ray_results
# 3.
#     Prints the devices in use, deleted since global tf import is disabled
#     from tensorflow.python.client import device_lib
#         print(device_lib.list_local_devices())
#         print("Tensorflow version is")
#         print(tf.__version__)
#         print("Keras version is")
#         print(keras.__version__)
# 4.
#     To use only GPU3, run commands using:
#         with tf.device('/device:XLA_GPU:3'):
# 5.
#     To use tensorboard, enable extension and add logging folder production:
#         import datetime
#         %load_ext tensorboard
#         log_dir="tensorboard-logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [2]:
# Importing Keras 
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.layers import Lambda
from keras import backend as K
from keras.engine.topology import Layer
from keras.layers.merge import concatenate
from keras.utils import plot_model
from keras.utils.io_utils import HDF5Matrix # NECESSARY
from keras.layers import SpatialDropout2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import Concatenate
print('Keras version: ' + keras.__version__)

# Import basic libraries
import numpy as np
import math
import random
import os
import re
import matplotlib
import time
import pickle 
import shutil

# Import HDF5 to use disk for large dataset
import h5py

import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.pyplot import cm
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.collections import PatchCollection

import scipy
import scipy.optimize as opt
from scipy.interpolate import griddata
from scipy import interpolate
from scipy.optimize import least_squares

# Import local libraries
from substructure import * # Jet substructure variables
import save_and_load

Using TensorFlow backend.


Keras version: 2.2.4


# ANALYSIS FUNCTIONS

In [3]:
def generate_real_SIC(expect,predict,masses,quality=1,verbose=False):
    if background_weight<0:
        raise Exception('Negative background weight. Please update .npy file!')
    if signal_weight<0:
        raise Exception('Negative signal weight. Please update .npy file!')
        
    to_sort = np.flip(np.array(sorted(np.vstack((expect,predict.flatten(),masses)).transpose(), key=lambda x: x[1])),0)
    efficiency = []; signal_eff = [];
    total_signal = np.sum(to_sort[:,0])
    for i in range(int(0.05*len(to_sort)),len(to_sort),quality*10):
        background_mass_binned,bins = np.histogram(to_sort[:i+1,2][to_sort[:i+1,0]==0],bins=np.arange(50,197,7))
        signal_mass_binned,bins = np.histogram(to_sort[:i+1,2][to_sort[:i+1,0]==1],bins=np.arange(50,197,7))
        log_likelihood = 0; baseLL = 0; 
        signal_eff.append(np.sum(to_sort[:i+1,0])/total_signal)
        test_LL_vs_SS = []
        for signal_strength in np.arange(1,5,quality/1000):
            log_likelihood = 0;
            for k in range(len(bins)-1):
                expected = background_weight*background_mass_binned[k]+signal_weight*signal_strength*signal_mass_binned[k]
                observed = background_weight*background_mass_binned[k]+signal_weight*signal_mass_binned[k]
                if (expected <= 0):
                    pass
                else:
                    log_likelihood = log_likelihood + observed*math.log(expected) - expected
            if signal_strength == 1:
                baseLL = log_likelihood
            test_LL_vs_SS.append(log_likelihood)
            if log_likelihood < baseLL-1/2:
                efficiency.append(1/(signal_strength-1))
                break
            if signal_strength > 3:
                efficiency.append(1/2)
                break
    max_eff = np.max(efficiency)
    max_cut = to_sort[efficiency.index(max_eff),1]
    print("base efficiency : " + str(float(efficiency[-1])))
    efficiency = np.array(efficiency)/float(efficiency[-1])
    max_eff = np.max(efficiency)
    print("Max SI of " + str(max_eff) + " at cut " + str(max_cut))
    return(efficiency,signal_eff)

def find_highest_SIC(expect,predict,quality=100,verbose=False):
    to_sort = np.flip(np.array(sorted(np.vstack((expect,predict.flatten())).transpose(), key=lambda x: x[1])),0)
    total_signal = np.sum(to_sort[:,0])
    efficiency = []
    for i in range(int(0.05*len(to_sort)),len(to_sort)): # generate a int range scanning over 95% of all samples??
        signal_eff_temp = np.sum(to_sort[:i+1,0])/total_signal
        background_eff_temp = (i+1-np.sum(to_sort[:i+1,0]))/(len(to_sort)-total_signal)
        efficiency.append((signal_eff_temp)/((background_eff_temp)**(1/2)))
    max_eff = np.max(efficiency)
    return(max_eff)

def find_highest_SIC_binned(expect,predict,masses):
    if background_weight<0:
        raise Exception('Negative background weight. Please update .npy file!')
    if signal_weight<0:
        raise Exception('Negative signal weight. Please update .npy file!')
        
    bins = np.arange(50,197,7)
    efficiency = []    
    sigmas = []
    for i in np.arange(0,0.9,0.05):
        #res = least_squares(lambda x : log_like(x,masses[np.logical_and(predict.flatten() >= i,expect == 0)],
        #                                        masses[np.logical_and(predict.flatten() >= i,expect == 1)]),x0=1)

        #i is the cut on the machine learning
        kept_back = masses[np.logical_and(predict.flatten() >= i,expect == 0)]
        kept_signal = masses[np.logical_and(predict.flatten() >= i,expect == 1)]
        j_array = []
        for j in np.arange(1,25,0.5):
            #print log_like(j,kept_back,kept_signal)
            if log_like(j,kept_back,kept_signal) > log_like(1,kept_back,kept_signal)+0.5:
                j_array.append(j-1)
                break
            if j >20:
                j_array.append(20)
                break
        print(j_array,i)
        sigmas.append(1/np.min(j_array))
        
    max_eff = np.max(sigmas)
    return(max_eff)

def generateSIC(expect,predict,quality=100,verbose=False):
    to_sort = np.flip(np.array(sorted(np.vstack((expect,predict.flatten())).transpose(), key=lambda x: x[1])),0)
    total_signal = np.sum(to_sort[:,0])
    efficiency = []; signal_eff = []
    for i in range(int(0.05*len(to_sort)),len(to_sort)):
        signal_eff_temp = np.sum(to_sort[:i+1,0])/total_signal
        background_eff_temp = (i+1-np.sum(to_sort[:i+1,0]))/(len(to_sort)-total_signal)
        signal_eff.append(signal_eff_temp)
        efficiency.append((signal_eff_temp)/((background_eff_temp)**(1/2)))
    max_eff = np.max(efficiency)
    max_cut = to_sort[efficiency.index(max_eff),1]
    print("Max SI of " + str(max_eff) + " at cut " + str(max_cut))
    return(efficiency,signal_eff)

def log_like(signal_strength,background_mass_list,signal_mass_list):
    if background_weight<0:
        raise Exception('Negative background weight. Please update .npy file!')
    if signal_weight<0:
        raise Exception('Negative signal weight. Please update .npy file!')
        
    log_likelihood = 0
    background_mass_binned,bins = np.histogram(background_mass_list,bins=np.arange(50,197,7))
    signal_mass_binned,bins = np.histogram(signal_mass_list,bins=np.arange(50,197,7))
    for i in range(len(bins)-1):
        expected = background_weight*background_mass_binned[i]+signal_weight*signal_strength*signal_mass_binned[i]
        observed = background_weight*background_mass_binned[i]+signal_weight*signal_mass_binned[i]
        if (expected <= 0):
            return float("inf")
        log_likelihood = log_likelihood + observed*math.log(expected) - expected
    return -log_likelihood

def log_like(signal_strength):
    if background_weight<0:
        raise Exception('Negative background weight. Please update .npy file!')
    if signal_weight<0:
        raise Exception('Negative signal weight. Please update .npy file!')
        
    log_likelihood = 0
    for i in range(len(bins)-1):
        expected = background_weight*background_mass_binned[i]+signal_weight*signal_strength*signal_mass_binned[i]
        observed = background_weight*background_mass_binned[i]+signal_weight*signal_mass_binned[i]
        #print(expected)
        log_likelihood = log_likelihood + observed*math.log(expected) - expected
    return log_likelihood

# Neural network utilities

In [4]:
#FRANK# Returns a function that generates a padding layer
#FRANK# x is the detector image, of following format:
#FRANK# [index, image#(charged and neutral pt and multiplicity), 40, 40]
def return_pad_me(padding):
    def pad_me(x):
        import tensorflow as tf # This solves 'TypeError: can't pickle _LazyLoader objects'
        #FRANK# x[:,:,:y,:] slice x off from y at the given axis.
        return(tf.concat((x,x[:,:,:padding,:]),2))
    return(pad_me)

def pad_out(padding,input_shape):
    return input_shape

class gen_call(Callback):
    
    def __init__(self, test_data):
        self.x, self.y = test_data
    
    def on_train_begin(self,logs={}):
        self.highest_SIC_train = []
        self.highest_SIC_test = []
        
    def on_epoch_end(self,epoch,logs={}):
        y_pred = self.model.predict(self.x)
        self.highest_SIC_test.append(find_highest_SIC(self.y,y_pred))
        print(str(self.highest_SIC_test[-1]) + " is how good")

def show_outputs(output):
    #Assumes the output is in shape like (32,41,36)
    
    fig = plt.figure(figsize=(8,6))
    
    for i in range(1,1+output.shape[0]):
        fig.add_subplot(4,output.shape[0]/4,i)
        plt.imshow(10*output[i-1,:,:])
        plt.axis('off')
    #plt.axis('off')
    plt.show()

In [5]:
#FRANK# this method is the exact replica of find_highest_SIC() but operates on tensors.
#FRANK# for some reason, pred has different format from expect. 
def highest_SIC_metric(y_true,y_pred):
    import tensorflow as tf # This solves 'TypeError: can't pickle _LazyLoader objects'
    print('highest_SIC_metric() is called')
    y_true = tf.keras.backend.flatten(y_true) #FRANK# flattens to 1D
    y_pred = tf.keras.backend.flatten(y_pred) #FRANK# flattens to 1D
    
    #stacked = tf.transpose(tf.stack((expect,predict)))
    #to_sort = tf.reverse(sorted(stacked, key=lambda x: x[1]),0) #FRANK# sorted is wrong!!
    total_sample = tf.cast(tf.size(y_pred), tf.float32)  # count total nums, then cast to float32 to avoid issue
    total_signal = tf.cast(tf.reduce_sum(y_true), tf.float32)  # summing across all predicted (where sigs are 1's and bkgs are 0's) to get total num of signal. 
    total_background = tf.cast(tf.subtract(total_sample, total_signal), tf.float32)  # subtracting signal countss from total size to get background counts
    
    
    # original mechanism:
    # 1. sort by ML score
    # 2. sum all actuals before an index
    # 3. count all 0's before an index 
    
    sorted_indices = tf.argsort(y_pred,axis=-1,direction='ASCENDING') # tf.argsort: Returns the indices of a tensor that give its sorted order along an axis.
    sorted_sigs = tf.gather(y_pred,sorted_indices)
    # return the indices of prediction in ascending order. By reading these indices, you can access corresponding expected y's.
    ones = tf.fill(tf.shape(sorted_sigs), 1.0)
    sorted_bkgs = tf.subtract(ones, sorted_sigs)
    # make a sorted tensor where 1's are bkgs and 0's are sigs
    
    sig_cum_sums = tf.cast(tf.cumsum(sorted_sigs), tf.float32) # return the integrated signal number from 0 to each index
    bkg_cum_sums = tf.cast(tf.cumsum(sorted_bkgs), tf.float32) # return the integrated bkg number from 0 to each index

    sig_effs = tf.divide(sig_cum_sums, total_signal)
    bkg_effs = tf.divide(bkg_cum_sums, total_background) #FRANK# total_background might be 0, causing bkg_effs=inf
    effs = tf.divide(sig_effs, tf.sqrt(bkg_effs))
        
    return tf.reduce_max(effs)

# Event image hyperparameter tuning

This section tunes the learning rate, epoch # and batch size of event image model

In [6]:
# Create Tune callback for Keras
class TuneReporterCallback(Callback):

    def __init__(self, logs={}):
        self.iteration = 0
        super(TuneReporterCallback, self).__init__()

    def on_epoch_end(self, batch, logs={}):
        self.iteration += 1
        tune.track.log(keras_info=logs, mean_accuracy=logs.get("accuracy"), mean_loss=logs.get("loss"))



In [7]:
# First define the global parameter input_shape - dimension of event images. 
# It is supposed to be a tuple.
# This parameter is stored in every event image hdf5 files. 
f = h5py.File('event-image-hdf5/train.hdf5', 'r')
input_shape = np.zeros(3, dtype='int32')
dset = f['input_shape']
dset.read_direct(input_shape)
input_shape=tuple(input_shape)
print(input_shape)

(3, 40, 40)


In [8]:
# Jan 22th version
# Parameters to tune:
#     Learning rate;
#     Batch size;

# Tensorboard callback is disabled since it is included in tune
#    import tensorflow as tf # This solves 'TypeError: can't pickle _LazyLoader objects'
#    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Create event image model automatically with tunable hyperparameter
def create_event_image_model(lr):
    import tensorflow as tf # This solves 'TypeError: can't pickle _LazyLoader objects'
    # --------------------------------------------------------------------------------------------
    # Event image model
    # --------------------------------------------------------------------------------------------
    event_image_cnn = Sequential()
    event_image_cnn.add(Lambda(return_pad_me(4),
                     input_shape=input_shape))
    event_image_cnn.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                     activation='relu',
                    data_format='channels_first'))
    event_image_cnn.add(Lambda(return_pad_me(1),
                     input_shape=input_shape))
    event_image_cnn.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2),data_format='channels_first'))
    event_image_cnn.add(Lambda(return_pad_me(4),
                     input_shape=input_shape))
    event_image_cnn.add(Conv2D(64, (5, 5), 
                     activation='relu',
                     data_format='channels_first'))
    event_image_cnn.add(Lambda(return_pad_me(1),
                     input_shape=input_shape))
    event_image_cnn.add(MaxPooling2D(pool_size=(2, 2),data_format='channels_first'))
    event_image_cnn.add(Flatten())
    event_image_cnn.add(Dense(300, activation='relu'))
    event_image_cnn.add(Dense(1, activation='sigmoid'))
    event_image_cnn.summary()

    #model_opt = keras.optimizers.Adadelta(lr=2.0, rho=0.95, epsilon=None, decay=0.0)
    model_opt = keras.optimizers.Adadelta(lr=lr) # learning rate and decay rates are tunable

    event_image_cnn.compile(loss=keras.losses.binary_crossentropy,
                  optimizer=model_opt,
                  metrics=['accuracy', highest_SIC_metric])
    return event_image_cnn

# Create training call for Tune using EVENT IMAGE
# `config` should contain keys:
#     'lr': learning rate
#     'epoch': epoch number
#     'batch': batch size
# Change these two paths to specify which dataset to read from and where to store 
# their copies for each worker:

source_hdf5_folder = '/home/ffu/higgs-classifier/analysis/event-image-hdf5' # Event images, including images 
                                                                            # generated from ggh (sig) and qcd (bkgd).
temp_path = '/data0/users/frankfu/hbb-qcd-hdf5/'
    
def tune_event_image(config):
    model = create_event_image_model(lr=config['lr'])
    # This saves the top model. `accuracy` is only available in TF2.0.
    checkpoint_callback = ModelCheckpoint(
        "model.h5", monitor='accuracy', save_best_only=True) #, save_freq=2)
    
    
    # Create a temporary copy of the dataset
    # Make a new randomly named folder
    folder_id = str(random.randint(0, 100000))
    while os.path.exists(temp_path+folder_id):
        folder_id = str(random.randint(0, 100000))
    os.makedirs(temp_path+folder_id)
    
    # copy all files to the temp directory
    src_files = os.listdir(source_hdf5_folder)
    for file_name in src_files:
        full_file_name = os.path.join(source_hdf5_folder, file_name)
        if os.path.isfile(full_file_name):
            shutil.copy(full_file_name, temp_path+folder_id)
    
    
    import tensorflow as tf # This solves 'TypeError: can't pickle _LazyLoader objects'
    # Train the model   
    x_train = HDF5Matrix(temp_path+folder_id+'/train.hdf5', 'x_train')
    y_train = HDF5Matrix(temp_path+folder_id+'/train.hdf5', 'y_train')
    x_test = HDF5Matrix(temp_path+folder_id+'/test.hdf5', 'x_test')
    y_test = HDF5Matrix(temp_path+folder_id+'/test.hdf5', 'y_test')
    model.fit(
        x_train, 
        y_train, 
        validation_data=(
            x_test, 
            y_test),
        verbose=0, batch_size=config['batch'], epochs=config['epoch'], 
        callbacks=[checkpoint_callback, TuneReporterCallback()],
        shuffle='batch'
    )
    
    # Remove the temporary folder and its contents
    shutil.rmtree(temp_path+folder_id)

    return model

In [9]:
# Configure hyperparameter space for random search
hyperparameter_space = {
    'lr': tune.sample_from(lambda spec: 10**np.random.uniform(low=-3.0, high=-1.0)), # range should be 0.00001 to 0.1, log-like.
    'batch': tune.sample_from(lambda spec: int(np.random.uniform(low=50, high=1000))), # range should be 50 to 1000?
    'epoch': tune.sample_from(lambda spec: int(np.random.uniform(low=100, high=500))) # range should be 100 to 500?
}

# Config trial number
num_samples = 20

In [ ]:
# Initialize Ray with memory size 3000000000, since train and test's total size is 2655952088
np.random.seed(5) # Set random seed
ray.shutdown()  # Restart Ray defensively in case the ray connection is lost. 
ray.init(num_gpus=1, memory=25000000000)  # memory=3000000000, object_store_memory=3000000000,
! rm -rf ~/ray_results/tune_event_image # Clean previous logs

# Pinning object to resolve 'Connection reset by peer'. Not used after switching to HDF5. 
# x_train = pin_in_object_store(x_train)
# y_train = pin_in_object_store(y_train)
# x_test = pin_in_object_store(x_test)
# y_test = pin_in_object_store(y_test)

# loading dataset in the form of hdf5 dataset. 
# MIGHT NOT WORK WITH TUNE's MULTITHREADING FEATURE. TESTING AS OF 3:27, Jan 23

# Run tuning
# run with GPU3 is disabled
# with tf.device('/device:XLA_GPU:3'):
analysis = tune.run(
    tune_event_image, 
    verbose=1, 
    config=hyperparameter_space,
    num_samples=num_samples)


Trial name,status,loc,lr,epoch,batch,total time (s),loss
tune_event_image_9badff3e,RUNNING,128.3.135.139:2069,0.00277963,182,877,1007.34,0.723941
tune_event_image_9bae96e2,RUNNING,128.3.135.139:2083,0.0687419,344,513,970.205,0.787411
tune_event_image_9baeefa2,RUNNING,128.3.135.139:2074,0.0340264,218,542,924.516,0.784295
tune_event_image_9baf6450,RUNNING,128.3.135.139:2070,0.00237379,395,126,952.279,0.724168
tune_event_image_9bb01fd0,RUNNING,128.3.135.139:2080,0.00763165,451,200,965.852,0.725261
tune_event_image_9bb083da,RUNNING,128.3.135.139:2073,0.00353324,218,443,937.123,0.730691
tune_event_image_9bb171fa,RUNNING,128.3.135.139:2071,0.0180957,339,600,929.381,0.745956
tune_event_image_9bb24f62,RUNNING,128.3.135.139:2082,0.00340125,201,320,964.447,0.727647
tune_event_image_9bb39318,RUNNING,128.3.135.139:2084,0.00451989,166,186,980.098,0.729456
tune_event_image_9bb3f18c,RUNNING,128.3.135.139:2075,0.0846956,175,962,977.466,0.818799


In [ ]:
ray.shutdown()

# Models (Example, please do not use)

In [ ]:



# --------------------------------------------------------------------------------------------
# Jet image model
# --------------------------------------------------------------------------------------------
jet_image_cnn = Sequential()
jet_image_cnn.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                data_format='channels_first',input_shape=input_shape))
#jet_image_cnn.add(Dropout(0.5))
jet_image_cnn.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2),data_format='channels_first'))
jet_image_cnn.add(Conv2D(64, (5, 5), activation='relu',data_format='channels_first'))
#jet_image_cnn.add(Dropout(0.5))
jet_image_cnn.add(MaxPooling2D(pool_size=(2, 2),data_format='channels_first'))
jet_image_cnn.add(Flatten())
jet_image_cnn.add(Dense(1000, activation='relu'))
#jet_image_cnn.add(Dropout(0.2))
jet_image_cnn.add(Dense(1, activation='sigmoid'))
#jet_image_cnn.summary()

model_opt = keras.optimizers.Adadelta()

jet_image_cnn.compile(loss=keras.losses.binary_crossentropy,
              optimizer=model_opt,
              metrics=['accuracy', 'highest_SIC_test'])



# --------------------------------------------------------------------------------------------
# Combined model
# --------------------------------------------------------------------------------------------
#FRANK# input shape is calculated during splitting:
#FRANK# splitData(background_image_list,signal_image_list)
#FRANK# how is jet image taken?
event_image_branch = Sequential()

#FRANK# Lambda is from Keras.
event_image_branch.add(Lambda(return_pad_me(5), #FRANK# tf.concat((x,x[:,:,:5,:]),2). Concatenate means pasting together.
                            #FRANK# this layer slices off an edge of the tensor and paste it on the other side.
                 input_shape=input_shape))
event_image_branch.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                data_format='channels_first'))
event_image_branch.add(Lambda(return_pad_me(2)))
event_image_branch.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
event_image_branch.add(Lambda(return_pad_me(5),
                 input_shape=input_shape))
event_image_branch.add(Conv2D(64, (5, 5), activation='relu'))
#event_image_branch.add(Dropout(0.5))
event_image_branch.add(Lambda(return_pad_me(2)))
event_image_branch.add(MaxPooling2D(pool_size=(2, 2)))
event_image_branch.add(Flatten())
event_image_branch.add(Dense(300,activation='relu'))
#event_image_branch.add(Dropout(0.5))

jet_image_branch = Sequential()
jet_image_branch.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                      kernel_initializer='random_uniform',input_shape=input_shape_r, data_format="channels_first")) #FRANK# problemmatic. see debug note #1
#jet_image_branch.add(Dropout(0.5))
jet_image_branch.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
jet_image_branch.add(Conv2D(64, (5, 5), activation='relu'))
#jet_image_branch.add(Dropout(0.5))
jet_image_branch.add(MaxPooling2D(pool_size=(2, 2)))
jet_image_branch.add(Flatten())
jet_image_branch.add(Dense(300, activation='relu'))
#jet_image_branch.add(Dropout(0.5))



#FRANK# Merge layer is no longer supported.  
#combined_model = Sequential()
#combined_model.add(Merge([event_image_branch, jet_image_branch], mode = 'concat'))
#combined_model.add(Dense(1,activation='sigmoid'))

#FRANK# these are not models, but tensors. Model.input or Model.output are tensor pointers I think.
combined_model_tensor = Concatenate(axis=-1)([event_image_branch.output, jet_image_branch.output])   
combined_model_tensor = Dense(1,activation='sigmoid')(combined_model_tensor)
#FRANK# what you need to do is merging output of 300 and 300 into one.

#combined_model.summary()
combined_model = Model([event_image_branch.input,jet_image_branch.input], combined_model_tensor)
combined_model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy', highest_SIC_metric])

# Visualizing event image model
plot_model(combined_model, show_shapes=True)

In [ ]:
# Launch Tensorboard (run without "%" in terminal)
%tensorboard --logdir tensorboard-logs/fit

In [ ]:
#FRANK# Sample training call testing metric
epochs = 100
print(y_train)
with tf.device('/device:XLA_GPU:3'):
    history_event_only = event_image_cnn.fit(
        x_train, 
        y_train,
        batch_size=1024,#batch_size,
        epochs=epochs,
        validation_data=(x_val,y_val),
        verbose=1, shuffle=True, callbacks=[
            tensorboard_callback, 
            EarlyStopping(monitor='highest_SIC_test', patience=2), 
            gen_call((x_val,y_val))])

#FRANK# for event image SIC: predicting
with tf.device('/device:XLA_GPU:3'):
    y_pred_test = model.predict(x_test)
    
print(y_pred_test)

In [ ]:
#FRANK# testing metric
print(np.array(y_pred.flatten()))
print(np.array(y_test))
print(highest_SIC_metric(y_pred,y_test))

In [ ]:
#FRANK# training history plots
def loss_history_plot(training_history):
    print(training_history.history.keys())
    plt.plot(training_history.history['loss'])
    plt.title('model accuracy')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()


# Evaluation

In [ ]:
# Print groomed mass histogram
plt.rcParams['figure.figsize'] = [7.5,6]

hist_bkg, bin_edges_bkg = np.histogram(background_mass_list, bins=21, range = (50, 197))
bin_centers_bkg = (bin_edges_bkg[:-1] + bin_edges_bkg[1:]) / 2
hist_sig, bin_edges_bkg = np.histogram(signal_mass_list, bins=21, range = (50, 197))


if background_weight<0:
    raise Exception('Negative background weight. Please update .npy file!')
if signal_weight<0:
    raise Exception('Negative signal weight. Please update .npy file!')
plt.step(bin_centers_bkg, hist_bkg*background_weight, label='bkgd')
plt.step(bin_centers_bkg, hist_sig*signal_weight, label='sig')
plt.legend()
plt.ylim(top=10000)
plt.ylim(bottom=0)
plt.xlim(right=197)  # adjust the right leaving left unchanged
plt.xlim(left=50) 
plt.show()
#log_like(1,background_mass_list=background_mass_list,signal_mass_list=signal_mass_list)

# Calculating beta3
background_beta3 = find_new_var_beta_3(background_reclustered,background_event_list_clustered,pt_cut = 1)
signal_beta3 = find_new_var_beta_3(signal_reclustered,signal_event_list_clustered,pt_cut = 1)

# Beta-3 histogram

plt.rcParams['figure.figsize'] = [5, 4]
beta3_min = min([min(background_beta3),min(signal_beta3)])
beta3_max = max([max(background_beta3),max(signal_beta3)])
print('beta3 range is '+str((beta3_min, beta3_max)))
plt.hist(background_beta3, label='bkgd', bins = 40, range = (beta3_min, 70), stacked=True, density=True,histtype ='step')
plt.hist(signal_beta3, label='sig', bins = 40, range = (beta3_min, 70), stacked=True, density=True,histtype ='step')
plt.xlabel("Beta-3")
plt.ylabel("Normalized Shape")
plt.legend()
plt.show()

plt.hist(background_mass_list, label='bkgd', bins = 21, range = (50, 197), stacked=True, density=True,histtype ='step')
plt.hist(signal_mass_list, label='sig', bins = 21, range = (50, 197), stacked=True, density=True,histtype ='step')
plt.xlabel("Trimmed Mass")
plt.ylabel("Normalized Shape")
plt.legend()
plt.show()

# Beta-3 plots
plot_simple_ROC_SIC(signal_beta3, background_beta3, step = 0.001):

In [ ]:
# Split the predicted results into sample and background using known answers.
# Here we assume all backgrounds are put before signals in all datasets. 
#TODO# Check before use!

def split_sig_bkgd(predicted, expected)
    split_location = int(np.argwhere(expected==1.)[0])
    #FRANK# signal predictions with event image model
    sig = predicted[split_location:]
    bkgd = predicted[:split_location]

    return sig, bkgd

In [ ]:
#FRANK# calculating true positive rates and false positive rates. 
#FRANK# TPR = TP/P
#FRANK# FPR = FP/N
#FRANK# note that bkgds have lower beta in general, so the cut should be 
#FRANK# beta3 > threshold.
#FRANK# 'where' usage referred from 
#FRANK# https://stackoverflow.com/questions/12995937/count-all-values-in-a-matrix-greater-than-a-value

def simple_ROC(sig, bkgd, step = 1):
    thres_min = min([min(sig),min(bkgd)])
    thres_max = max([max(sig),max(bkgd)])
    search_range=(thres_min, thres_max)
    sig_count = len(sig)
    bkgd_count = len(bkgd)
    sig_rates = []
    bkgd_rates = []
    thres_list = np.arange(search_range[0], search_range[1], step)
    for thres in thres_list:
        sig_selected = np.greater(sig, thres)
        bkgd_selected = np.greater(bkgd, thres)
        sig_rates.append(np.sum(sig_selected)/sig_count)
        bkgd_rates.append(np.sum(bkgd_selected)/bkgd_count)
    return sig_rates, bkgd_rates, thres_list

def plot_simple_ROC_SIC(sig, bkgd, step = 0.001):
    
    # Calculate simple ROC curve
    sig_rates, bkgd_rates, thres_list = simple_ROC(sig, bkgd, step)
    

    # Plotting ROC
    plt.rcParams['figure.figsize'] = [6, 6]
    plt.plot(sig_rates, bkgd_rates,label='bkgd',linewidth=1)
    plt.legend()
    plt.xlabel("signal rate")
    plt.ylabel("background rate")
    plt.show()

    # Plotting SIC (ε/√ε_b) 
    event_significances = sig_rates/np.sqrt(bkgd_rates)
    plt.rcParams['figure.figsize'] = [5, 4]
    plt.plot(sig_rates,event_significances,label='beta3',linewidth=1)
    plt.legend()
    plt.xlabel("signal rate")
    plt.ylabel("significance improvement")
    plt.ylim(top=2.5)
    plt.ylim(bottom=1)
    plt.xlim(right=1)  # adjust the right leaving left unchanged
    plt.xlim(left=0.1) 
    plt.show()

In [ ]:
#Here evaluate ML performance by e.g.
#r_combine_y,r_combine_x = generate_real_SIC(y_test,new_model_combine.predict([x_test,x_test_r]),mass_test,quality=1)

r_full_y,r_full_x = generateSIC(y_test,predicted, quality=100)
#r_fine_y,r_fine_x = generate_real_SIC(y_test,model_fine.predict(x_test_r),mass_test,quality=1)

In [ ]:
plt.figure(figsize=(10,7))
plt.title("Binned Likelihood Significance Improvements for various Architectures")
#plt.plot(r_combine_x,r_combine_y,color="red",label="Full CNN Architecture",linewidth=1)
plt.plot(r_full_x,r_full_y,color="red",alpha=0.6,label="Event image only",linewidth=1,linestyle="dashed")
#plt.plot(r_fine_x,r_fine_y,color="red",alpha=0.6,label="Jet image only",linewidth=1,linestyle="dotted")
#plt.plot(beta_x,beta_y,color="blue",alpha=0.6,label=r"$\beta_3$",linewidth=1)#,linestyle="dashed")
#plt.plot(three_x,three_y,color="blue",alpha=0.6,label=r"$Rb_2$",linewidth=1,linestyle="dotted")
#plt.plot(x_02,y_02,color="gray",label="Jet image, no neutral layer",linewidth=1)
#plt.plot(vars_y,vars_x,color="blue",label=r"$\beta_3 + Rb_2$",linewidth=1)
plt.xlim(0.1,1)
plt.ylim(1,2.3)
plt.legend()
plt.xlabel("Signal Efficiency")
plt.ylabel("Significance Improvement")
plt.savefig('all_SIC.eps')
plt.show()

In [ ]:
plt.figure(figsize=(9,7))
plt.title("Binned Likelihood Significance Improvements for various Architectures")
#plt.plot(r_combine_x,r_combine_y,color="red",label="Full CNN Architecture",linewidth=1)
#plt.plot(r_full_x,r_full_y,color="red",alpha=0.6,label="Event image only",linewidth=1,linestyle="dashed")
#plt.plot(r_fine_x,r_fine_y,color="red",alpha=0.6,label="Jet image only",linewidth=1,linestyle="dotted")
#FRANK# event image SIC
plt.plot(r_full_x,r_full_y,color="red",alpha=0.6,label="Event image only",linewidth=1,linestyle="dashed")
plt.plot(sig_rates,significances,label='Beta3',linewidth=1)
#FRANK# beta-3 SIC
#plt.plot(beta_x,beta_y,color="blue",alpha=0.6,label=r"$\beta_3$",linewidth=1)#,linestyle="dashed")
#plt.plot(three_x,three_y,color="blue",alpha=0.6,label=r"$Rb_2$",linewidth=1,linestyle="dotted")
#plt.plot(x_02,y_02,color="gray",label="Jet image, no neutral layer",linewidth=1)
#plt.plot(vars_y,vars_x,color="blue",label=r"$\beta_3 + Rb_2$",linewidth=1)
plt.xlim(0.1,1)
plt.ylim(1,2.5)
plt.legend()
plt.xlabel("Signal Efficiency")
plt.ylabel("Significance Improvement")
plt.savefig('all_SIC.eps')
plt.show()